In [ ]:
# default_exp evaluation.core

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

# Evaluation

> API details.

In [ ]:
# export
import re

import numpy as np
import pandas as pd
import tensorflow as tf

from collections import Counter, defaultdict
from icodegen.data.core import replace_spec_toks_to_original, java_special_tokens
from icodegen.data.transforms import (
    code_token_randomizer,
    line_randomizer,
    java_comment_remover,
    transform_df,
)
from icodegen.model.core import Model, RNNModel
from pathlib import Path
from scipy import stats
from tqdm.auto import tqdm
from typing import Dict, List, Optional

In [ ]:
# hide
# Setting up testing data
from transformers import GPT2TokenizerFast, TFGPT2LMHeadModel
from icodegen.model.core import RNNModel

# Using tiny-gpt2 for just quick tests since it is... tiny :)
trnsfr_tokenizer = GPT2TokenizerFast.from_pretrained("sshleifer/tiny-gpt2")
tokenizer = trnsfr_tokenizer.backend_tokenizer
trnsfr = TFGPT2LMHeadModel.from_pretrained("sshleifer/tiny-gpt2")
# trnsfr_model = TransformerModel(tokenizer, trnsfr)

rnn_type = "gru"
n_layers = 1
vocab_size = tokenizer.get_vocab_size()
embedding_dim = 128
rnn_units = 128
batch_size = 1
out_path = "/tmp"
gru_model = RNNModel(
    rnn_type,
    n_layers,
    vocab_size,
    embedding_dim,
    rnn_units,
    batch_size,
    out_path,
    tokenizer,
)

df_fake = pd.DataFrame(
    ["aaaa(bb(aaaa(bb()()ccc)dd)()ccc)dd", "aaaa(bb()ccccc)dd"], columns=["code"]
)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at sshleifer/tiny-gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
# export
def get_mean_probs(df: pd.DataFrame, model: Model, n: Optional[int] = None):
    """
    Get the mean probability of each token that the model
    should predict for an entire pandas dataframe.

    :param df: the pandas dataframe containing each method to have the model predict on
    :param model: the model used to generate the predictions
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a numpy array of the mean probability for each token in the model's vocab
    """
    if n is None:
        n = len(df)

    # setup container lists for the number of occurrences and sum of probabilities for each token
    counts = [0] * model.tokenizer.get_vocab_size()
    sum_probs = [0.0] * model.tokenizer.get_vocab_size()
    # loop through each method
    for mthd in df.code.values[:n]:
        # token the method and generate the probabilities for the model's predictions
        inputs = model.tokenize(mthd)
        probs = model.get_probs(inputs)[0].numpy()

        # loop through each token and its probability and update the container lists
        for idx, p in zip(inputs["input_ids"][0], probs):
            counts[idx] += 1
            sum_probs[idx] += p[idx]

    # convert the lists to numpy lists and perform element wise division to get the mean probabilities for each token
    counts = np.array(counts)
    sum_probs = np.array(sum_probs)

    # perform division, but not when denominator is zero. In those cases, just leave value as NAN.
    nans = np.empty(counts.shape)
    nans.fill(np.nan)
    mean_probs = np.divide(sum_probs, counts, out=nans, where=counts != 0)
    # TODO: convert to dictionary with keys as tokens
    mean_probs = {
        model.tokenizer.id_to_token(i): mean_probs[i] for i in range(len(mean_probs))
    }
    return mean_probs

In [ ]:
NON_NAN_PROBS_MEAN = np.array(
    [
        2.01237513e-05,
        1.98944481e-05,
        2.01449202e-05,
        2.04353437e-05,
        2.02043060e-05,
        2.02826177e-05,
        2.09888076e-05,
        2.07051467e-05,
        1.98100976e-05,
        2.02152678e-05,
        2.02035244e-05,
        2.10283021e-05,
    ]
)

mean_probs = np.array(list(get_mean_probs(df_fake, trnsfr_model).values()))
non_nan_idx = np.argwhere(~np.isnan(mean_probs)).flatten()
non_nan_mean_prob = mean_probs[non_nan_idx]

assert np.isclose(non_nan_mean_prob, NON_NAN_PROBS_MEAN, atol=1.0e-6).all()

In [ ]:
NON_NAN_PROBS_MEAN = np.array(
    [
        1.99270412e-05,
        1.99168703e-05,
        1.98815596e-05,
        1.99057849e-05,
        1.98800869e-05,
        1.98893995e-05,
        1.98797388e-05,
        1.98960342e-05,
        1.99086674e-05,
        1.98605580e-05,
        1.98807957e-05,
        1.98842057e-05,
    ]
)

mean_probs = np.array(list(get_mean_probs(df_fake, gru_model).values()))
non_nan_idx = np.argwhere(~np.isnan(mean_probs)).flatten()
non_nan_mean_prob = mean_probs[non_nan_idx]

assert np.isclose(non_nan_mean_prob, NON_NAN_PROBS_MEAN, atol=1.0e-6).all()

In [ ]:
# export
def find_parens(toks: List[str], opening: str, closing: str) -> Dict[int, int]:
    """
    Get the indices for the opening and closing tokens.
    From https://stackoverflow.com/a/29992065/5768407
    by user Baltasarq (https://stackoverflow.com/users/266978/baltasarq).

    :param toks: the tokenized version of a method
    :param opening: the opening token that will be matched against the closing token
    :param closing: the closing token that will be matched against the opening token
    :returns: returns a dictionary with the opening token indices as the keys and the closing token indices as the values
    """
    toret = {}
    pstack = []

    for i, tok in enumerate(toks):
        if tok == opening:
            pstack.append(i)
        elif tok == closing:
            if len(pstack) == 0:
                raise IndexError("No matching closing parens at: " + str(i))
            toret[pstack.pop()] = i

    if len(pstack) > 0:
        raise IndexError("No matching opening parens at: " + str(pstack.pop()))

    return toret


def _get_dist_probs(
    mthd: str, model: Model, opening: str, closing: str
) -> Dict[int, float]:
    """
    Get the distances and mean probabilities between opening and closing tokens in a given method.

    :param mthd: the method to get the ranges of the opening and closing tokens and their probabilities
    :param model: the model used to generate the predictions
    :param opening: the opening token used for calculating the distance between opening and closing tokens
    :param closing: the closing token used for calculating the distance between opening and closing tokens as well as the token to get the mean probability of
    :returns: returns a dictionary with the distance between the opening and closing tokens as keys and their mean probabilities as values
    """
    # WARNING: Careful when using different tokenizers since HF tokenizers lib have diff API then HF transformers lib tokenizers... You will need to update this when using custom model and tokenizer...

    # get the distances for the opening and closing tokens
    toks = model.tokenizer.encode(mthd).tokens
    idxs = find_parens(toks, opening, closing)

    # get the model probabilities for the given method
    inputs = model.tokenize(mthd)
    probs = model.get_probs(inputs)[0].numpy()

    # sum up the probabilities of the different distances for the closing token
    dist_probs = defaultdict(float)
    for open_id, close_id in idxs.items():
        dist_probs[close_id - open_id] += probs[close_id][
            inputs["input_ids"][0][close_id]
        ]

    # get the mean of the summed probabilities
    dist_cnts = Counter([close_id - open_id for open_id, close_id in idxs.items()])
    dist_probs = {dist: dist_probs[dist] / n for dist, n in dist_cnts.items()}
    return dist_probs


def mean_dist_probs(
    df: pd.DataFrame,
    model: Model,
    opening: Optional[str] = "<{>",
    closing: Optional[str] = "<}>",
    n: Optional[int] = None,
) -> pd.DataFrame:
    """
    Get the distance between opening and closing tokens and the mean probability of each closing token that the model should predict for an entire pandas dataframe.

    :param df: the pandas dataframe containing each method to have the model predict on
    :param model: the model used to generate the predictions
    :param opening: the opening token used for calculating the distance between opening and closing tokens
    :param closing: the closing token used for calculating the distance between opening and closing tokens as well as the token to get the mean probability of
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a dataframe with the distances between opening and closing tokens and their mean probabilities
    """
    if n is None:
        n = len(df)

    # get the probabilities for the different distances for an entire dataframe
    df = df.iloc[:n].copy()
    dist_probs = df.code.apply(
        lambda mthd: _get_dist_probs(mthd, model, opening, closing)
    ).values

    # flatten the keys of the different distances into a list
    dist_keys = []
    for probs in dist_probs:
        dist_keys.extend(probs.keys())
    # merge dictionaries across methods by taking the mean of probs with the same distance. Modified from https://stackoverflow.com/a/10461916/5768407,
    # users georg https://stackoverflow.com/users/989121/georg and Rémy Hosseinkhan Boucher https://stackoverflow.com/users/12149730/r%c3%a9my-hosseinkhan-boucher
    mean_dist_probs = {
        k: np.nanmean(np.array([probs.get(k, np.nan) for probs in dist_probs]))
        for k in set(dist_keys)
    }
    std_dist_probs = {
        k: np.nanstd(np.array([probs.get(k, np.nan) for probs in dist_probs]))
        for k in set(dist_keys)
    }

    med_dist_probs = {
        k: np.nanmedian(np.array([probs.get(k, np.nan) for probs in dist_probs]))
        for k in set(dist_keys)
    }
    mad_dist_probs = {
        k: stats.median_abs_deviation(
            np.array([probs.get(k, np.nan) for probs in dist_probs]), nan_policy="omit"
        )
        for k in set(dist_keys)
    }
    # TODO: convert to dictionary
    df_dist = (
        pd.DataFrame(
            {
                "dist": list(mean_dist_probs.keys()),
                "mean_prob": list(mean_dist_probs.values()),
                "std_prob": list(std_dist_probs.values()),
                "med_prob": list(med_dist_probs.values()),
                "mad_prob": list(mad_dist_probs.values()),
            }
        )
        .sort_values("dist")
        .reset_index(drop=True)
    )
    return df_dist

In [ ]:
DIST_DF = pd.DataFrame(
    {
        "dist": [6, 10, 16],
        "mean_prob": [
            1.98822217e-05,
            1.97613608e-05,
            1.97816771e-05,
        ],
        "std_prob": [
            4.93400876e-09,
            0.00000000e00,
            0.00000000e00,
        ],
        "med_prob": [
            2.04683793e-05,
            2.07205376e-05,
            1.97817026e-05,
        ],
        "mad_prob": [
            4.93400876e-09,
            0.00000000e00,
            0.00000000e00,
        ],
    }
)
df_dist = mean_dist_probs(df_fake, gru_model, opening="(", closing=")")

assert (DIST_DF.dist.values == df_dist.dist.values).all()
assert np.isclose(DIST_DF.mean_prob.values, df_dist.mean_prob.values, atol=1.0e-6).all()
assert np.isclose(DIST_DF.std_prob.values, df_dist.std_prob.values, atol=1.0e-6).all()
assert np.isclose(DIST_DF.med_prob.values, df_dist.med_prob.values, atol=1.0e-6).all()
assert np.isclose(DIST_DF.mad_prob.values, df_dist.mad_prob.values, atol=1.0e-6).all()

In [ ]:
DIST_DF = pd.DataFrame(
    {
        "dist": [6, 10, 16],
        "mean_prob": [
            1.98822217e-05,
            1.97613608e-05,
            1.97816771e-05,
        ],
        "std_prob": [
            4.93400876e-09,
            0.00000000e00,
            0.00000000e00,
        ],
        "med_prob": [
            2.04683793e-05,
            2.07205376e-05,
            1.97817026e-05,
        ],
        "mad_prob": [
            4.93400876e-09,
            0.00000000e00,
            0.00000000e00,
        ],
    }
)
df_dist = mean_dist_probs(df_fake, trnsfr_model, opening="(", closing=")")

assert (DIST_DF.dist.values == df_dist.dist.values).all()
assert np.isclose(DIST_DF.mean_prob.values, df_dist.mean_prob.values, atol=1.0e-6).all()
assert np.isclose(DIST_DF.std_prob.values, df_dist.std_prob.values, atol=1.0e-6).all()
assert np.isclose(DIST_DF.med_prob.values, df_dist.med_prob.values, atol=1.0e-6).all()
assert np.isclose(DIST_DF.mad_prob.values, df_dist.mad_prob.values, atol=1.0e-6).all()

In [ ]:
# export
token_taxonomy = {
  "blocks": {
    "<{>": "{",
    "<}>": "}",
    "<[>": "[",
    "<]>": "]",
    "<(>": "(",
    "<)>": ")",
    "<;>": ";",
    "<return>": "return"
  },
  "exceptions": {
    "<catch>": "catch",
    "<try>": "try",
    "<finally>": "finally",
    "<throw>": "throw",
    "<throws>": "throws"
  },
  "oop": {
    "<class>": "class",
    "<instanceof>": "instanceof",
    "<interface>": "interface",
    "<private>": "private",
    "<protected>": "protected",
    "<public>": "public",
    "<abstract>": "abstract",
    "<extends>": "extends",
    "<package>": "package",
    "<this>": "this",
    "<implements>": "implements",
    "<import>": "import",
    "<new>": "new",
    "<super>": "super"
  },
  "tests": {
    "<assert>": "assert"
  },
  "declarations": {
    "<native>": "native",
    "<static>": "static",
    "<synchronized>": "synchronized",
    "<transient>": "transient",
    "<volatile>": "volatile",
    "<void>": "void",
    "<final>": "final",
    "<enum>": "enum"
  },
  "conditionals": {
    "<else>": "else",
    "<if>": "if",
    "<switch>": "switch",
    "<case>": "case",
    "<default>": "default"
  },
  "loops": {
    "<break>": "break",
    "<do>": "do",
    "<for>": "for",
    "<while>": "while",
    "<continue>": "continue"
  },
  "operators": {
    "<=>": "=",
    "<+>": "+",
    "<->": "-",
    "<*>": "*",
    "</>": "/",
    "<%>": "%",
    "<++>": "++",
    "<-->": "--",
    "<!>": "!",
    "<==>": "==",
    "<!=>": "!=",
    "<greater_equal>": ">=",
    "<lesser_equal>": "<=",
    "<&&>": "&&",
    "<||>": "||",
    "<?>": "?",
    "<:>": ":",
    "<~>": "~",
    "<double_lesser>": "<<",
    "<double_greater>": ">>",
    "<triple_greater>": ">>>",
    "<&>": "&",
    "<^>": "^",
    "<|>": "|"
  },
  "datatypes": {
    "<byte>": "byte",
    "<char>": "char",
    "<float>": "float",
    "<boolean>": "boolean",
    "<double>": "double",
    "<int>": "int",
    "<long>": "long",
    "<short>": "short",
    "<strictfp>": "strictfp"
  },
  "extra_tokens": {
    "<@>": "@",
    "<...>": "...",
    "<null>": "null",
    "<true>": "true",
    "<false>": "false",
    "<n>": "\n"
  }
}

In [ ]:
# export
non_wordy = ["<n>", "<...>", "<@>", *token_taxonomy["operators"], *token_taxonomy["blocks"]]
non_wordy.remove("<return>")

In [ ]:
# export
ERROR_THRESHOLD = 0.5

def get_error_rates(df: pd.DataFrame, model: Model, n: Optional[int] = None):
    if n is None:
        n = len(df)

    # setup container lists for the number of occurrences and sum of probabilities for each token
    cnts = [0] * model.tokenizer.get_vocab_size()
    err_cnts = [0] * model.tokenizer.get_vocab_size()
    # loop through each method
    for mthd in df.code.values[:n]:
        # token the method and generate the probabilities for the model's predictions
        inputs = model.tokenize(mthd)
        probs = model.get_probs(inputs)[0].numpy()

        # loop through each token and its probability and update the container lists
        for idx, p in zip(inputs["input_ids"][0], probs):
            cnts[idx] += 1
            if p[idx] < ERROR_THRESHOLD:
                err_cnts[idx] += 1

    # convert the lists to numpy lists and perform element wise division to get the mean probabilities for each token
    cnts = np.array(cnts)
    err_cnts = np.array(err_cnts)

    # perform division, but not when denominator is zero. In those cases, just leave value as NAN.
    nans = np.empty(cnts.shape)
    nans.fill(np.nan)
    mean_errs = np.divide(err_cnts, cnts, out=nans, where=cnts != 0)
    
    error_taxonomy = token_taxonomy.copy()
    
    for cat, tokens in error_taxonomy.items():
        errs = []
        cnt_sum = 0
        for token, keyword in tokens.items():
            idx = model.tokenizer.token_to_id(token)
            error_taxonomy[cat][token] = {"error_rate": mean_errs[idx], "count": cnts[idx]}
            errs.append(mean_errs[idx])
            cnt_sum += cnts[idx]

        errs = np.array(errs)
        error_taxonomy[cat]["stats"] = {
            "mean_error_rate": np.nanmean(errs),
            "stdev_error_rate": np.nanstd(errs),
            "median_error_rate": np.nanmedian(errs),
            "mad_error_rate": stats.median_abs_deviation(errs, nan_policy="omit"),
        }
    
    return error_taxonomy

In [ ]:
bugfix_path = Path("/home/jovyan/work/dvc-icodegen/datasets/controlled/testbeds/_ts_bug_fix")
df_buggy = pd.read_json(bugfix_path / "buggy.jsonl", orient="records", lines=True)[
    :10
]
model = RNNModel.from_path("/home/jovyan/work/dvc-icodegen/models/gru_layers1_vocab10000_embed256_units512")
err_tax = get_error_rates(df_buggy, model)
err_tax

{'blocks': {'<{>': {'error_rate': 1.0, 'count': 27},
  '<}>': {'error_rate': 1.0, 'count': 27},
  '<[>': {'error_rate': 1.0, 'count': 3},
  '<]>': {'error_rate': 1.0, 'count': 3},
  '<(>': {'error_rate': 1.0, 'count': 81},
  '<)>': {'error_rate': 0.8888888888888888, 'count': 81},
  '<;>': {'error_rate': 1.0, 'count': 53},
  '<return>': {'error_rate': 1.0, 'count': 10},
  'stats': {'mean_error_rate': 0.9861111111111112,
   'stdev_error_rate': 0.03674654598700822,
   'median_error_rate': 1.0,
   'mad_error_rate': 0.0}},
 'exceptions': {'<catch>': {'error_rate': 1.0, 'count': 1},
  '<try>': {'error_rate': 1.0, 'count': 1},
  '<finally>': {'error_rate': nan, 'count': 0},
  '<throw>': {'error_rate': nan, 'count': 0},
  '<throws>': {'error_rate': 1.0, 'count': 1},
  'stats': {'mean_error_rate': 1.0,
   'stdev_error_rate': 0.0,
   'median_error_rate': 1.0,
   'mad_error_rate': 0.0}},
 'oop': {'<class>': {'error_rate': 1.0, 'count': 1},
  '<instanceof>': {'error_rate': nan, 'count': 0},
  '<in

In [ ]:
# export
ERROR_THRESHOLD = 0.5

def get_error_rates_df(df: pd.DataFrame, model: Model, bs: int = 16, n: Optional[int] = None):
    if n is None:
        n = len(df)

    # setup container lists for the number of occurrences and sum of probabilities for each token
    rows = []
    # loop through each method
    for i in tqdm(range(0, n, bs), desc="Error Rates", total = (n // bs) + 1):
        batch = ["<sos>" + mthd for mthd in df.code.values[i:i + bs]]
        # token the method and get the probabilities for each token from the model
        inputs = tf.stack([x.ids for x in model.tokenizer.encode_batch(batch)], axis = 0)
        logits = model.model(inputs)
        probs = tf.nn.softmax(logits).numpy()
        
        for i in range(len(batch)):
            row = {
                "y_" + k: np.array([0.] * model.tokenizer.get_vocab_size())
                for k in token_taxonomy.keys()
            }
            row_cnt = {
                "y_" + k: np.array([0] * model.tokenizer.get_vocab_size())
                for k in token_taxonomy.keys()
            }
            # loop through each token and its probability and update the container lists
            for j, (idx, p) in enumerate(zip(inputs[i], probs[i])):
                if p[idx] < ERROR_THRESHOLD:
                    tok = model.tokenizer.id_to_token(idx)
                    for k in token_taxonomy:
                        if tok in token_taxonomy[k]:
                            # Check if token is wordy and could be part of variable or method
                            if tok not in non_wordy:
                                # Get the token version of the token behind the token under study
                                # and check if the last character in the token contains a letter
                                inp_tok_prev = model.tokenizer.id_to_token(inputs[i][j - 1])
                                if re.search('[a-zA-Z]', inp_tok_prev[-1]):
                                    break
                                # Check if there is a token infront of the token under study
                                # if there is, get the token version of it
                                # and check if the first character in the token contains a letter
                                if j + 1 < len(inputs[i]):
                                    inp_tok_next = model.tokenizer.id_to_token(inputs[i][j + 1])
                                    if re.search('[a-zA-Z]', inp_tok_next[0]):
                                        break
                            row["y_" + k][idx] += p[idx]
                            row_cnt["y_" + k][idx] += 1

            for k in row:
                # Check if there were no tokens found in this method for this particular taxonomy category
                if not row_cnt[k].any():
                    row[k] = np.nan
                else:
                    sum_cnt = np.sum(row_cnt[k])
                    row[k] = np.sum(row[k]) / sum_cnt

            rows.append(row)
        
    error_df = pd.DataFrame(rows)
    error_df["original_code"] = replace_spec_toks_to_original(df, java_special_tokens, n).code.values
    error_df["transformed_code"] = df.code.values[:n]
    
    return error_df

In [ ]:
bugfix_path = Path("/home/jovyan/work/dvc-icodegen/datasets/controlled/testbeds/_ts_bug_fix")
df_buggy = pd.read_json(bugfix_path / "buggy.jsonl", orient="records", lines=True)[
    :100
]
model = RNNModel.from_path("/home/jovyan/work/dvc-icodegen/models/controlled/rnns/rnn_layers1_vocab10000_embed256_units1024")

In [ ]:
err_tax = get_error_rates_df(df_buggy, model)
err_tax.describe()

,y_blocks,y_exceptions,y_oop,y_tests,y_declarations,y_conditionals,y_loops,y_operators,y_datatypes,y_extra_tokens
count,100.000000,2.800000e+01,9.800000e+01,0.0,6.400000e+01,57.000000,1.900000e+01,9.700000e+01,4.100000e+01,100.000000
mean,0.030134,6.314229e-06,1.747945e-03,NaN,1.479936e-06,0.000023,8.963920e-06,1.727996e-03,1.199283e-05,0.130223
std,0.019572,7.585470e-06,3.444690e-03,NaN,8.380155e-06,0.000025,6.504990e-06,5.010473e-03,1.776130e-05,0.039746
min,0.002529,1.614634e-08,1.236387e-08,NaN,3.552259e-08,0.000005,4.059797e-07,6.955367e-07,2.637347e-08,0.010788
25%,0.016589,1.111324e-06,8.302875e-08,NaN,1.712802e-07,0.000009,4.271065e-06,1.845865e-04,8.623591e-08,0.103158
50%,0.025835,2.241509e-06,1.344181e-05,NaN,3.463187e-07,0.000013,7.019739e-06,4.883553e-04,3.648138e-06,0.130483
75%,0.040499,1.035260e-05,2.141599e-03,NaN,3.845131e-07,0.000025,1.417009e-05,1.127912e-03,1.580466e-05,0.158202
max,0.087365,2.462625e-05,2.073873e-02,NaN,6.730417e-05,0.000137,1.905133e-05,4.579088e-02,7.690572e-05,0.224735


In [ ]:
err_tax.head()

,y_blocks,y_exceptions,y_oop,y_tests,y_declarations,y_conditionals,y_loops,y_operators,y_datatypes,y_extra_tokens,transformed_code,original_code
0,0.016703,NaN,8.925746e-06,NaN,3.519077e-07,NaN,NaN,0.000315,NaN,0.189852,<private> <void> success<(>io.netty.channel.Ch...,private void success(io.netty.channel.Channel ...
1,0.019511,NaN,1.236387e-08,NaN,3.519077e-07,0.00001,NaN,0.004352,NaN,0.131604,<private> <void> handleConnectRequest<(>com.as...,private void handleConnectRequest(com.assistan...
2,0.014283,NaN,1.757487e-04,NaN,3.093981e-07,NaN,NaN,0.000488,2.802167e-05,0.162338,<@>java.lang.Override<n><protected> <void> onS...,@java.lang.Override\nprotected void onSizeChan...
3,0.025020,NaN,8.302875e-08,NaN,NaN,0.00001,0.000006,0.000996,2.637347e-08,0.097829,<public> <boolean> <do>esEdgesHaveWeight<(><)>...,public boolean doesEdgesHaveWeight() {\n if...
4,0.070037,NaN,2.955031e-04,NaN,1.820177e-07,NaN,NaN,0.000131,NaN,0.124276,<@>org.junit.Test<n><public> <void> configures...,@org.junit.Test\npublic void configuresMultipl...


In [ ]:
# export
def get_mean_cross_entropy(df: pd.DataFrame, model: Model, n: Optional[int] = None):
    """
    Get the mean cross entropy for a model on an entire pandas dataframe

    :param df: the pandas dataframe containing each method to have the model predict on
    :param model: the model used to generate the predictions
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns the mean cross entropy of the models predictions compared to true labels
    """
    if n is None:
        n = len(df)

    cross_entropy_losses = []
    # Need to change to sparse_categorical_crossentropy
    for mthd in df.code.values[:n]:
        # token the method and get the probabilities for each token from the model
        inputs = model.tokenize(mthd)
        probs = model.get_probs(inputs)[0].numpy()

        # calculate the cross entropy between the labels and probabilities
        losses = tf.keras.losses.sparse_categorical_crossentropy(
            inputs["input_ids"], probs
        ).numpy()
        cross_entropy_losses.append(losses)

    # flatten list of cross entropies and calculate the mean, median, std, and mad
    cross_entropy_losses = np.concatenate(cross_entropy_losses)
    return {
        "mean": np.mean(cross_entropy_losses),
        "median": np.median(cross_entropy_losses),
        "std": np.std(cross_entropy_losses),
        "mad": stats.median_abs_deviation(cross_entropy_losses),
    }

In [ ]:
cross_entropy_losses = []
for mthd in df_fake.code.values:
    inputs = gru_model.tokenize(mthd)
    probs = gru_model.get_probs(inputs)[0].numpy()

    losses = tf.keras.losses.sparse_categorical_crossentropy(
        inputs["input_ids"], probs
    ).numpy()
    cross_entropy_losses.append(losses)

CROSS_ENTROPY_MEAN = np.mean(np.concatenate(cross_entropy_losses))
CROSS_ENTROPY_MEDIAN = np.median(np.concatenate(cross_entropy_losses))
CROSS_ENTROPY_STD = np.std(np.concatenate(cross_entropy_losses))
CROSS_ENTROPY_MAD = stats.median_abs_deviation(np.concatenate(cross_entropy_losses))
cross_entropy = get_mean_cross_entropy(df_fake, gru_model)

assert np.isclose(CROSS_ENTROPY_MEAN, cross_entropy["mean"], atol=1.0e-6)
assert np.isclose(CROSS_ENTROPY_MEDIAN, cross_entropy["median"], atol=1.0e-6)
assert np.isclose(CROSS_ENTROPY_STD, cross_entropy["std"], atol=1.0e-6)
assert np.isclose(CROSS_ENTROPY_MAD, cross_entropy["mad"], atol=1.0e-6)

In [ ]:
cross_entropy_losses = []
for mthd in df_fake.code.values:
    inputs = trnsfr_model.tokenize(mthd)
    probs = trnsfr_model.get_probs(inputs)[0].numpy()

    losses = tf.keras.losses.sparse_categorical_crossentropy(
        inputs["input_ids"], probs
    ).numpy()
    cross_entropy_losses.append(losses)

CROSS_ENTROPY_MEAN = np.mean(np.concatenate(cross_entropy_losses))
CROSS_ENTROPY_MEDIAN = np.median(np.concatenate(cross_entropy_losses))
CROSS_ENTROPY_STD = np.std(np.concatenate(cross_entropy_losses))
CROSS_ENTROPY_MAD = stats.median_abs_deviation(np.concatenate(cross_entropy_losses))
cross_entropy = get_mean_cross_entropy(df_fake, trnsfr_model)

assert np.isclose(CROSS_ENTROPY_MEAN, cross_entropy["mean"], atol=1.0e-6)
assert np.isclose(CROSS_ENTROPY_MEDIAN, cross_entropy["median"], atol=1.0e-6)
assert np.isclose(CROSS_ENTROPY_STD, cross_entropy["std"], atol=1.0e-6)
assert np.isclose(CROSS_ENTROPY_MAD, cross_entropy["mad"], atol=1.0e-6)

In [ ]:
# export
def get_mean_probs(df: pd.DataFrame, model: Model, n: Optional[int] = None):
    """
    Get the mean probability of each token that the model
    should predict for an entire pandas dataframe.

    :param df: the pandas dataframe containing each method to have the model predict on
    :param model: the model used to generate the predictions
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a numpy array of the mean probability for each token in the model's vocab
    """
    if n is None:
        n = len(df)

    # setup container lists for the number of occurrences and sum of probabilities for each token
    counts = [0] * model.tokenizer.get_vocab_size()
    sum_probs = [0.0] * model.tokenizer.get_vocab_size()
    # loop through each method
    for mthd in df.code.values[:n]:
        # token the method and generate the probabilities for the model's predictions
        inputs = model.tokenize(mthd)
        probs = model.get_probs(inputs)[0].numpy()

        # loop through each token and its probability and update the container lists
        for idx, p in zip(inputs["input_ids"][0], probs):
            counts[idx] += 1
            sum_probs[idx] += p[idx]

    # convert the lists to numpy lists and perform element wise division to get the mean probabilities for each token
    counts = np.array(counts)
    sum_probs = np.array(sum_probs)

    # perform division, but not when denominator is zero. In those cases, just leave value as NAN.
    nans = np.empty(counts.shape)
    nans.fill(np.nan)
    mean_probs = np.divide(sum_probs, counts, out=nans, where=counts != 0)
    # TODO: convert to dictionary with keys as tokens
    mean_probs = {
        model.tokenizer.id_to_token(i): mean_probs[i] for i in range(len(mean_probs))
    }
    return mean_probs

In [ ]:
# export
def get_mean_cross_entropy_df(df: pd.DataFrame, model: Model, bs = 16, n: Optional[int] = None):
    """
    Get the mean cross entropy for a model on an entire pandas dataframe

    :param df: the pandas dataframe containing each method to have the model predict on
    :param model: the model used to generate the predictions
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns the mean cross entropy of the models predictions compared to true labels
    """
    if n is None:
        n = len(df)

    cross_entropy_losses = []
    for i in tqdm(range(0, n, bs), desc="Cross Entropies", total = (n // bs) + 1):
        batch = ["<sos>" + mthd for mthd in df.code.values[i:i + bs]]
        # token the method and get the probabilities for each token from the model
        inputs = tf.stack([x.ids for x in model.tokenizer.encode_batch(batch)], axis = 0)
        logits = model.model(inputs)
        probs = tf.nn.softmax(logits).numpy()

        # calculate the cross entropy between the labels and probabilities
        losses = tf.keras.losses.sparse_categorical_crossentropy(
            inputs, probs
        ).numpy()
        cross_entropy_losses.extend(np.mean(losses, axis = 1))
    
    new_df = pd.DataFrame(
        zip(replace_spec_toks_to_original(df, java_special_tokens, n).code.values, df.code.values[:n], cross_entropy_losses),
        columns=["original_code", "transformed_code", "y_cross_entropy"]
    )

    return new_df

In [ ]:
from glob import glob

path = Path("/home/jovyan/work/dvc-icodegen/nbs/nbs_experiments/results/analyses")
for d in glob(str(path) + "/*/"):
    if "gru_layers3_vocab10000_embed256_units1024" in d: continue
    print(d)
    bug_fix_cross_entropy = pd.read_json(
        d + "/bug_fix_cross_entropy.jsonl", orient="records", lines=True
    )
    bug_fix_cross_entropy["original_code"] = replace_spec_toks_to_original(
        bug_fix_cross_entropy, java_special_tokens
    ).code.values
    bug_fix_cross_entropy.rename(columns={"code": "transformed_code"}, inplace=True)
    bug_fix_cross_entropy.to_json(d + "/bug_fix_cross_entropy.jsonl", orient="records", lines=True)
    
#     bug_fix_error_taxonomy = pd.read_json(
#         d + "/bug_fix_error_taxonomy.jsonl", orient="records", lines=True
#     )
#     bug_fix_error_taxonomy["original_code"] = replace_spec_toks_to_original(
#         bug_fix_error_taxonomy, java_special_tokens
#     ).code.values
#     bug_fix_error_taxonomy.rename(columns={"code": "transformed_code"}, inplace=True)

/home/jovyan/work/dvc-icodegen/nbs/nbs_experiments/results/analyses/gru_layers1_vocab10000_embed256_units2048/
/home/jovyan/work/dvc-icodegen/nbs/nbs_experiments/results/analyses/gru_layers2_vocab10000_embed256_units1024/
/home/jovyan/work/dvc-icodegen/nbs/nbs_experiments/results/analyses/rnn_layers1_vocab10000_embed256_units1024/
/home/jovyan/work/dvc-icodegen/nbs/nbs_experiments/results/analyses/gru_layers1_vocab10000_embed256_units512/


In [ ]:
bugfix_path = Path("/home/jovyan/work/dvc-icodegen/datasets/controlled/testbeds/_ts_bug_fix")
df_buggy = pd.read_json(bugfix_path / "buggy.jsonl", orient="records", lines=True)[
    :100
]
model = RNNModel.from_path("/home/jovyan/work/dvc-icodegen/models/controlled/rnns/rnn_layers1_vocab10000_embed256_units1024")

In [ ]:
cross_entropy = get_mean_cross_entropy_df(df_buggy, model)
cross_entropy.describe()

,y_cross_entropy
count,100.000000
mean,6.036340
std,1.529498
min,3.113172
25%,4.934342
50%,5.905706
75%,6.866514
max,9.877866


In [ ]:
cross_entropy.head()

,original_code,transformed_code,y_cross_entropy
0,private void success(io.netty.channel.Channel ...,<private> <void> success<(>io.netty.channel.Ch...,6.006714
1,private void handleConnectRequest(com.assistan...,<private> <void> handleConnectRequest<(>com.as...,6.368460
2,@java.lang.Override\nprotected void onSizeChan...,<@>java.lang.Override<n><protected> <void> onS...,3.927362
3,public boolean doesEdgesHaveWeight() {\n if...,<public> <boolean> <do>esEdgesHaveWeight<(><)>...,3.711157
4,@org.junit.Test\npublic void configuresMultipl...,<@>org.junit.Test<n><public> <void> configures...,6.497345


In [ ]:
# export
_TRANSFORMs = {
#     "randomized_tokens": code_token_randomizer,
#     "randomized_lines": line_randomizer,
    "comments_removed": java_comment_remover,
}

In [ ]:
# export
def _get_metrics(df, model):
#     mean_probs = get_mean_probs(df, model)
    error_taxonomy_df = get_error_rates_df(df, model, bs = 192)
#     df_dist = mean_dist_probs(df, model)
#     mean_cross_entropy_df = get_mean_cross_entropy_df(df, model, bs = 192)

    return {
        "error_taxonomy": error_taxonomy_df,
#         "dist_mean": df_dist,
#         "mean_cross_entropy": mean_cross_entropy_df,
    }


def _long_range(bigclone_path, bugfix_path, codesearchnet_path, model, out_path, n=None):
    out_path.mkdir(parents=True, exist_ok=True)
    long_range_results = {}

    # TODO add bigclone data

#     df_buggy = pd.read_json(bugfix_path / "buggy.jsonl", orient="records", lines=True)[
#         :n
#     ]
#     buggy_metrics = _get_metrics(df_buggy, model)

#     df_fixed = pd.read_json(bugfix_path / "fixed.jsonl", orient="records", lines=True)[
#         :n
#     ]
#     fixed_metrics = _get_metrics(df_fixed, model)
    
#     bug_fix_err_df = pd.concat(
#         [buggy_metrics["error_taxonomy"], fixed_metrics["error_taxonomy"]]
#     ).sort_index().reset_index(drop=True)
#     bug_fix_err_df["x_treatment"] = [False, True] * len(buggy_metrics["error_taxonomy"])
#     bug_fix_err_df.to_json(out_path / "bug_fix_error_taxonomy.jsonl", orient="records", lines=True)
    
#     bug_fix_cross_df = pd.concat(
#         [buggy_metrics["mean_cross_entropy"], fixed_metrics["mean_cross_entropy"]]
#     ).sort_index().reset_index(drop=True)
#     bug_fix_cross_df["x_treatment"] = [False, True] * len(buggy_metrics["mean_cross_entropy"])
#     bug_fix_cross_df.to_json(out_path / "bug_fix_cross_entropy.jsonl", orient="records", lines=True)

#     df_codesearchnet = pd.read_json(
#         codesearchnet_path / "codesearchnet_java" / "test.jsonl",
#         orient="records",
#         lines=True,
#     )[:n]
#     long_range_results["codesearchnet_original"] = _get_metrics(df_codesearchnet, model)

#     for transform in _TRANSFORMs:
#         df_transformed = transform_df(df_codesearchnet, _TRANSFORMs[transform])
#         long_range_results["codesearchnet_" + transform] = _get_metrics(
#             df_transformed, model
#         )

    return long_range_results


def _counterfactual(control_results, treatment_results):
    pass


def evaluate(data_path, model_path, experiment_path):
    """Function for evaluating models related to the library."""
    results = defaultdict(dict)
    testbed_path = data_path / "testbed"
    #     models = []
    # These model folders will need to contain the config of the model as well
    # to differentiate them
    for m_path in model_path.glob("*/"):
        model = None
        if "dvc" in m_path.name:
            continue
        elif ".gitignore" in m_path.name:
            continue
        print(m_path)
        model = RNNModel.from_path(m_path)
#         if m_path.name == "Transformer":
#             model = TransformerModel.from_path(m_path)
#         elif "rnn" in m_path.name:
#             model = RNNModel.from_path(m_path)
#         elif m_path.name == "RNN":
#             pass
#         return model
    
        bigclone_path = testbed_path / "ts-bigclone-types"
        bugfix_path = testbed_path / "ts-bug-fix"
        codesearchnet_path = testbed_path / "ts-comments"

        # Long-Range Interactions
        _long_range(
            bigclone_path, bugfix_path, codesearchnet_path,
            model, experiment_path / m_path.name
        )
        # Counterfactuals

In [ ]:
bugfix_path = Path("/home/jovyan/work/dvc-icodegen/testbed/ts-bug-fix")
df_buggy = pd.read_json(bugfix_path / "buggy.jsonl", orient="records", lines=True)
df_fixed = pd.read_json(bugfix_path / "fixed.jsonl", orient="records", lines=True)

In [ ]:
len(df_buggy), len(df_fixed)

(64722, 64722)

In [ ]:
path = Path("/home/jovyan/work")
data_path = path / "dvc-icodegen"
model_path = path / "dvc-icodegen/models/controlled/rnns/"
experiment_path = path / "dvc-icodegennbs/nbs_experiments/results/analyses"
results = evaluate(data_path, model_path, experiment_path)

/home/jovyan/work/dvc-icodegen/models/controlled/rnns/rnn_layers1_vocab10000_embed256_units1024



/home/jovyan/work/dvc-icodegen/models/controlled/rnns/gru_layers1_vocab10000_embed256_units1024



/home/jovyan/work/dvc-icodegen/models/controlled/rnns/gru_layers1_vocab10000_embed256_units512


In [ ]:
bug_fix_cross_df = pd.read_json(
    experiment_path / "rnn_layers1_vocab10000_embed256_units1024/bug_fix_cross_entropy.jsonl", orient="records", lines=True
)

In [ ]:
bug_fix_cross_df.head(10)

,code,y_cross_entropy,x_treatment
0,<private> <void> success<(>io.netty.channel.Ch...,6.006220,False
1,<private> <void> success<(>io.netty.channel.Ch...,6.008038,True
2,<private> <void> handleConnectRequest<(>com.as...,6.368741,False
3,<private> <void> handleConnectRequest<(>com.as...,6.337473,True
4,<@>java.lang.Override<n><protected> <void> onS...,3.927699,False
5,<@>java.lang.Override<n><protected> <void> onS...,3.910351,True
6,<public> <boolean> <do>esEdgesHaveWeight<(><)>...,3.711497,False
7,<public> <boolean> <do>esEdgesHaveWeight<(><)>...,4.510262,True
8,<@>org.junit.Test<n><public> <void> configures...,6.497474,False
9,<@>org.junit.Test<n><public> <void> configures...,5.749438,True


In [ ]:
error_taxonomy_df = pd.read_json(
    experiment_path / "rnn_layers1_vocab10000_embed256_units1024/bug_fix_error_taxonomy.jsonl", orient="records", lines=True
)

In [ ]:
error_taxonomy_df.head(10)

,blocks,exceptions,oop,tests,declarations,conditionals,loops,operators,datatypes,extra_tokens,code,x_treatment
0,0.0027,0.0,0.0002,0.0000,0.0001,0.0000,0.0001,0.0003,0.0000,0.0008,<private> <void> success<(>io.netty.channel.Ch...,False
1,0.0027,0.0,0.0002,0.0000,0.0001,0.0000,0.0001,0.0003,0.0000,0.0008,<private> <void> success<(>io.netty.channel.Ch...,True
2,0.0027,0.0,0.0001,0.0000,0.0001,0.0002,0.0000,0.0002,0.0000,0.0012,<private> <void> handleConnectRequest<(>com.as...,False
3,0.0027,0.0,0.0001,0.0000,0.0001,0.0002,0.0000,0.0002,0.0000,0.0012,<private> <void> handleConnectRequest<(>com.as...,True
4,0.0018,0.0,0.0002,0.0000,0.0001,0.0000,0.0000,0.0006,0.0005,0.0009,<@>java.lang.Override<n><protected> <void> onS...,False
5,0.0018,0.0,0.0002,0.0000,0.0001,0.0000,0.0000,0.0006,0.0005,0.0009,<@>java.lang.Override<n><protected> <void> onS...,True
6,0.0030,0.0,0.0001,0.0000,0.0000,0.0002,0.0002,0.0005,0.0001,0.0013,<public> <boolean> <do>esEdgesHaveWeight<(><)>...,False
7,0.0029,0.0,0.0001,0.0000,0.0000,0.0002,0.0002,0.0005,0.0001,0.0012,<public> <boolean> <do>esEdgesHaveWeight<(><)>...,True
8,0.0023,0.0,0.0003,0.0002,0.0001,0.0000,0.0001,0.0001,0.0000,0.0006,<@>org.junit.Test<n><public> <void> configures...,False
9,0.0023,0.0,0.0003,0.0002,0.0001,0.0000,0.0001,0.0001,0.0000,0.0006,<@>org.junit.Test<n><public> <void> configures...,True


In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()

Converted 00_data.core.ipynb.
Converted 01_data.transforms.ipynb.
Converted 02_model.core.ipynb.
Converted 04_evaluation.core.ipynb.
Converted index.ipynb.
